In [1]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [2]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [3]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        33
           1       0.98      0.98      0.98        54

    accuracy                           0.98        87
   macro avg       0.98      0.98      0.98        87
weighted avg       0.98      0.98      0.98        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [4]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.81      0.90        37
           1       0.88      1.00      0.93        50

    accuracy                           0.92        87
   macro avg       0.94      0.91      0.92        87
weighted avg       0.93      0.92      0.92        87



# Lets add data, fakenews!

In [5]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [6]:
data = pd.concat([fakenews, df], ignore_index=True)


In [7]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.78      0.87        40
           1       0.94      0.94      0.94        51
           2       0.80      0.96      0.87        46

    accuracy                           0.90       137
   macro avg       0.91      0.89      0.90       137
weighted avg       0.91      0.90      0.90       137



In [8]:
confusion_matrix(y_test, y_pred)

array([[31,  1,  8],
       [ 0, 48,  3],
       [ 0,  2, 44]], dtype=int64)

## Is the vectorizer that good?

In [9]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [10]:
data = pd.concat([bbc, data], ignore_index=True)

In [11]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [12]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.97      0.89      0.93        36
           2       0.75      0.68      0.71        22
           3       0.97      0.78      0.86        41
           4       0.76      0.88      0.81        32
           5       0.78      0.97      0.87        30
           6       0.94      0.91      0.92        32
           7       0.96      1.00      0.98        25

    accuracy                           0.88       233
   macro avg       0.88      0.89      0.88       233
weighted avg       0.89      0.88      0.88       233



In [13]:
confusion_matrix(y_test, y_pred)

array([[15,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 32,  1,  0,  1,  1,  1,  0],
       [ 1,  0, 15,  1,  3,  2,  0,  0],
       [ 0,  1,  1, 32,  3,  4,  0,  0],
       [ 0,  0,  2,  0, 28,  0,  1,  1],
       [ 0,  0,  1,  0,  0, 29,  0,  0],
       [ 0,  0,  0,  0,  2,  1, 29,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 25]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.

## Lets create a transformer

In [14]:
df_deeplearning = df_shuffled

In [15]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Use tokenizer to encode
df_deeplearning['input_ids'] = df_deeplearning['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

In [16]:
df_deeplearning

,cleaned_text,label,input_ids
0,Beer giant swallows Russian firm Brewing gian...,3,"[101, 5404, 5016, 26436, 2845, 3813, 16005, 50..."
1,Uganda bans Vagina Monologues Ugandas authori...,6,"[101, 10031, 7221, 2015, 12436, 20876, 18847, ..."
2,Davies favours Gloucester future Wales hooker...,7,"[101, 9082, 7927, 2015, 13370, 2925, 3575, 170..."
3,Elvis regains top chart position Elvis Presle...,6,"[101, 12280, 12452, 2015, 2327, 3673, 2597, 12..."
4,In a few moments Marianne Solomin Paul and two...,0,"[101, 1999, 1037, 2261, 5312, 19887, 3948, 100..."
...,...,...,...
769,Ferguson rues failure to cut gap Boss Sir Ale...,7,"[101, 11262, 13413, 2015, 4945, 2000, 3013, 65..."
770,Conquer Fortune he indeed did For in less than...,0,"[101, 16152, 7280, 2002, 5262, 2106, 2005, 199..."
771,Said the sinner Lord I hear thy voice but I se...,0,"[101, 2056, 1996, 26403, 2121, 2935, 1045, 296..."
772,Snow Patrol feted at Irish awards Snow Patrol...,6,"[101, 4586, 6477, 10768, 3064, 2012, 3493, 298..."


In [17]:
input_ids = torch.tensor(df_deeplearning['input_ids'].tolist(), dtype=torch.long)
labels = torch.tensor(df_deeplearning['label'].tolist(), dtype=torch.float)  # Assuming multi-label

# Create torch dataset
dataset = TensorDataset(input_ids, labels)

train_length=int(0.7* len(dataset))

test_length=len(dataset)-train_length

train_dataset, test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length))


train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)  
        return x
    

model_0 = SimpleModel(input_size=128, hidden_units=16, output_size=8)

In [19]:
input_ids.size()

torch.Size([774, 128])

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr = 0.01)

In [ ]:
def training_loop(loss_fn, optimizer, model,epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        train_loss = 0
        for batch, (X, y) in enumerate(train_dataloader):
            model.train()
            

            X, y = X.to(device), y.to(device)

            X = X.float() 
            y = y.long()  
            
            y_pred = model(X)

            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

        
        train_loss /= len(train_dataloader)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {train_loss:.4f}")


# Call the training loop
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_0, epochs=100)


Epoch 1/100, Average Training Loss: 98.7219
Epoch 21/100, Average Training Loss: 2.0798
Epoch 41/100, Average Training Loss: 2.0768
Epoch 61/100, Average Training Loss: 2.0778
Epoch 81/100, Average Training Loss: 2.0766


In [22]:
class MoreAdvancedModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, hidden_units)
        self.layer3 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)  
        return x
    

model_1 = MoreAdvancedModel(input_size=128, hidden_units=512, output_size=8)

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr = 0.01)

In [24]:
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_1, epochs=100)

Epoch 1/100, Average Training Loss: 2.0750
Epoch 21/100, Average Training Loss: 2.0748
Epoch 41/100, Average Training Loss: 2.0756
Epoch 61/100, Average Training Loss: 2.0749
Epoch 81/100, Average Training Loss: 2.0762


In [25]:
all_preds = []
true_preds = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1.to(device)
model_1.eval()
with torch.no_grad():
    for data in test_loader:
        inputs = data[0].to(device).float()  
        outputs = model_1(inputs)
        preds = torch.argmax(outputs, dim=1)  # or outputs if you want raw logits/probs
        all_preds.append(preds)
        true_preds.append(data[1])

true_preds = torch.cat(true_preds).tolist()
final_preds = torch.cat(all_preds).tolist()

In [26]:
from collections import defaultdict, Counter

# Group predictions by true label
label_predictions = defaultdict(list)

for pred, true in zip(final_preds, true_preds):
    label_predictions[true].append(pred)

# Now sort the predictions by frequency for each true label
sorted_preds_per_class = {
    label: Counter(preds).most_common()  # Returns list of (pred, count) sorted by count
    for label, preds in label_predictions.items()
}


In [27]:
sorted_preds_per_class

{5.0: [(3, 16), (7, 5), (1, 1), (4, 1), (6, 1)],
 4.0: [(3, 11), (7, 7), (5, 4), (1, 2), (0, 2), (2, 1)],
 7.0: [(3, 15), (7, 9), (6, 3), (1, 1), (0, 1), (5, 1)],
 0.0: [(7, 8), (3, 7), (1, 4), (0, 2), (4, 1)],
 2.0: [(7, 10), (3, 8), (6, 3), (1, 3), (5, 2)],
 6.0: [(3, 15), (7, 14), (6, 3), (5, 3), (0, 2)],
 3.0: [(3, 15), (7, 14), (5, 5), (4, 3), (6, 3), (1, 1), (0, 1)],
 1.0: [(3, 10), (7, 9), (6, 3), (5, 2), (4, 1)]}

Just guesses 7 :/